# Imports, leitura e estruturação dos dados

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import math
import datetime
from dateutil.relativedelta import relativedelta
import sqlalchemy

Neste notebook vou resolver um problema de previsão de perda de clientes, portanto o primeiro passo para tal é entender o negócio, ou seja, compreender fatores que levam o cliente a abandonar a empresa. Portanto, começarei pensando em dados relevantes encontrados no [dataset da olist no kaggle](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce), criarei minha ABT (analytical base table), farei a análise exploratória de dados e criarei meu modelo.

Variáveis que vou usar:
* olist_order_items_dataset(order_id, seller_id, product_id, price, *freight_value) *talvez vendedores com fretes mais altos tendem a abandonar a empresa
* olist_order_reviews_dataset(review_score, review_creation_date, review_answer_timestamp). Pretendo criar uma variável representando o tempo para resposta e outra indicando se houve resposta ou não.
* olist_orders_dataset(order_status, order_approved_at)
* olist_products_dataset(product_category_name). Pretendo criar variáveis, como variedade de produtos de um vendedor.

Além disso, manipularei esses dados para criar variáveis que segmentam os clientes através da classificação RFV (recência, frequência e valor), como quantidade de produtos vendidos, periodicidade do vendedor, ticket médio, etc.

In [2]:
FILE_PATH = os.path.abspath("__file__")
PROJECT_PATH = os.path.dirname(FILE_PATH)
DATA_PATH = os.path.join(PROJECT_PATH, 'data')

In [3]:
df_oi = pd.read_csv(os.path.join(DATA_PATH,'olist_order_items_dataset.csv'))
df_or = pd.read_csv(os.path.join(DATA_PATH,'olist_order_reviews_dataset.csv'))
df_orders = pd.read_csv(os.path.join(DATA_PATH,'olist_orders_dataset.csv'))
df_products = pd.read_csv(os.path.join(DATA_PATH,'olist_products_dataset.csv'))

In [4]:
df = pd.concat([df_oi, df_or, df_orders, df_products], axis=1)
df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,review_id,order_id,review_score,...,order_estimated_delivery_date,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4.0,...,2017-10-18 00:00:00,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5.0,...,2018-08-13 00:00:00,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5.0,...,2018-09-04 00:00:00,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5.0,...,2017-12-15 00:00:00,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5.0,...,2018-02-26 00:00:00,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [5]:
COLS_TO_USE = ['order_id', 'seller_id', 'product_id', 'price', 'freight_value','review_score', 'review_creation_date',
               'review_answer_timestamp', 'order_status', 'order_approved_at', 'order_delivered_carrier_date', 
               'order_delivered_customer_date', 'order_estimated_delivery_date','product_category_name']
df = df.loc[:,~df.columns.duplicated()][COLS_TO_USE]
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,review_creation_date,review_answer_timestamp,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_category_name
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,4244733e06e7ecb4970a6e2683c13e61,58.90,13.29,4.0,2018-01-18 00:00:00,2018-01-18 21:46:59,delivered,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,perfumaria
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,e5f2d52b802189ee658865ca93d83a8f,239.90,19.93,5.0,2018-03-10 00:00:00,2018-03-11 03:05:13,delivered,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,artes
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,c777355d18b72b67abbeef9df44fd0fd,199.00,17.87,5.0,2018-02-17 00:00:00,2018-02-18 14:36:24,delivered,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,esporte_lazer
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,7634da152a4610f1595efa32f14722fc,12.99,12.79,5.0,2017-04-21 00:00:00,2017-04-21 22:02:06,delivered,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,bebes
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,ac6c3623068f30de03045865e4e10089,199.90,18.14,5.0,2018-03-01 00:00:00,2018-03-02 10:26:53,delivered,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,utilidades_domesticas


In [6]:
DATE_COLS = ['review_creation_date', 'review_answer_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 
             'order_estimated_delivery_date']
df[DATE_COLS] = pd.to_datetime(df[DATE_COLS].stack()).unstack()
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,review_creation_date,review_answer_timestamp,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_category_name
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,4244733e06e7ecb4970a6e2683c13e61,58.90,13.29,4.0,2018-01-18,2018-01-18 21:46:59,delivered,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,perfumaria
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,e5f2d52b802189ee658865ca93d83a8f,239.90,19.93,5.0,2018-03-10,2018-03-11 03:05:13,delivered,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,artes
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,c777355d18b72b67abbeef9df44fd0fd,199.00,17.87,5.0,2018-02-17,2018-02-18 14:36:24,delivered,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,esporte_lazer
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,7634da152a4610f1595efa32f14722fc,12.99,12.79,5.0,2017-04-21,2017-04-21 22:02:06,delivered,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,bebes
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,ac6c3623068f30de03045865e4e10089,199.90,18.14,5.0,2018-03-01,2018-03-02 10:26:53,delivered,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,utilidades_domesticas


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 14 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       112650 non-null  object        
 1   seller_id                      112650 non-null  object        
 2   product_id                     112650 non-null  object        
 3   price                          112650 non-null  float64       
 4   freight_value                  112650 non-null  float64       
 5   review_score                   99224 non-null   float64       
 6   review_creation_date           99224 non-null   datetime64[ns]
 7   review_answer_timestamp        99224 non-null   datetime64[ns]
 8   order_status                   99441 non-null   object        
 9   order_approved_at              99281 non-null   datetime64[ns]
 10  order_delivered_carrier_date   97658 non-null   datetime64[ns]
 11  

In [8]:
df['order_status'].value_counts()

delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: order_status, dtype: int64

In [9]:
df = df.drop(df.loc[df['order_status'] != 'delivered'].index)

In [10]:
df['order_status'].value_counts()

delivered    96478
Name: order_status, dtype: int64

Próximos passos:
* Criar as novas variáveis.
    * quantidade de vendas
    * dias sem vender
    * frequência de vendas (quantidade de dias que um vendedor leva para vender novamente)
    * total de dinheiro em vendas
    * preço médio por venda
    * média de vendas mensais
    * média de vendas nos meses em que o vendedor estava ativo
    * etc.

# Criação de novas variáveis

In [24]:
engine = sqlalchemy.create_engine('sqlite:///olist.db', echo=True)
sqlite_connection = engine.connect()

In [26]:
sqlite_table = "Chrun_Prediction"
df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2022-06-05 19:01:13,372 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Chrun_Prediction")
2022-06-05 19:01:13,372 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-05 19:01:13,380 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Chrun_Prediction")
2022-06-05 19:01:13,380 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-05 19:01:13,380 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Chrun_Prediction" (
	"index" BIGINT, 
	order_id TEXT, 
	seller_id TEXT, 
	product_id TEXT, 
	price FLOAT, 
	freight_value FLOAT, 
	review_score FLOAT, 
	review_creation_date DATETIME, 
	review_answer_timestamp DATETIME, 
	order_status TEXT, 
	order_approved_at DATETIME, 
	order_delivered_carrier_date DATETIME, 
	order_delivered_customer_date DATETIME, 
	order_estimated_delivery_date DATETIME, 
	product_category_name TEXT
)


2022-06-05 19:01:13,380 INFO sqlalchemy.engine.Engine [no key 0.00120s] ()
2022-06-05 19:01:13,642 INFO sqlalchemy.engine.Engine COMMIT
2022-06-05 19:01:13,642 INFO sqlalchem

96478

In [19]:
MIN_DATE = datetime.datetime.strptime('01102016', "%d%m%Y").date()
print(MIN_DATE)

2016-10-01


In [22]:
MIN_DATE + relativedelta(months=+6)

datetime.date(2017, 4, 1)

In [11]:
def make_snapshot(initial_date, final_date, df):
    return df

In [12]:
def create_simple_group_col(new_col, col, agg):
    df[new_col] = df.groupby('seller_id')[col].transform(agg)

In [13]:
def create_ratio_col(new_col, dividend_col, divisor_col):
    df[new_col] = df.dividend_col/df.divisor_col

In [14]:
def create_op_by_month_col(new_col, col, date_col):
    df[new_col] = (df[col]/
                         round(((df[date_col].iloc[-1] - df[date_col].
                                 iloc[0])/np.timedelta64(1, 'M'))))

In [15]:
def create_day_diff_col(new_col, col1, col2):
    df[new_col] = (df[col1] - df[col2]).dt.days
    df[new_col] = df.groupby('seller_id')[new_col].transform('mean')

## Quantidade de vendas de cada vendedor

In [73]:
df['qtd_vendas'] = df.groupby('seller_id')['order_id'].transform('count')
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,review_creation_date,review_answer_timestamp,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_category_name,qtd_vendas
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,4244733e06e7ecb4970a6e2683c13e61,58.90,13.29,4.0,2018-01-18,2018-01-18 21:46:59,delivered,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,perfumaria,127
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,e5f2d52b802189ee658865ca93d83a8f,239.90,19.93,5.0,2018-03-10,2018-03-11 03:05:13,delivered,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,artes,117
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,c777355d18b72b67abbeef9df44fd0fd,199.00,17.87,5.0,2018-02-17,2018-02-18 14:36:24,delivered,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,esporte_lazer,13
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,7634da152a4610f1595efa32f14722fc,12.99,12.79,5.0,2017-04-21,2017-04-21 22:02:06,delivered,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,bebes,15
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,ac6c3623068f30de03045865e4e10089,199.90,18.14,5.0,2018-03-01,2018-03-02 10:26:53,delivered,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,utilidades_domesticas,22


In [74]:
df.loc[df['seller_id'] == '001cca7ae9ae17fb1caed9dfb1094831'].head(1)

,order_id,seller_id,product_id,price,freight_value,review_score,review_creation_date,review_answer_timestamp,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_category_name,qtd_vendas
176,006e43460a55bc60c0a437521e426529,001cca7ae9ae17fb1caed9dfb1094831,08574b074924071f4e201e151b152b4e,99.0,43.06,4.0,2017-07-25,2017-08-02 18:15:52,delivered,2018-04-04 08:50:15,2018-04-07 00:49:39,2018-04-16 23:35:26,2018-05-02,livros_interesse_geral,208


## Data de referência (final de cada snapshot)

In [75]:
LAST_DATE = df.sort_values(by='order_approved_at', 
                      ascending=False)['order_approved_at'].iloc[0] - pd.DateOffset(months=3)
LAST_DATE

Timestamp('2018-05-29 15:10:26')

In [76]:
df = df[df['order_approved_at'] < LAST_DATE]
df = df.sort_values(by=['seller_id', 'order_approved_at'])
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,review_creation_date,review_answer_timestamp,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_category_name,qtd_vendas
93696,d455a8cb295653b55abda06d434ab492,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,2018-07-28,2018-07-30 23:59:38,delivered,2016-10-10 16:02:37,2016-10-18 15:53:51,2016-10-26 16:35:46,2016-12-14,NaN,3
69082,9dc8d1a6f16f1b89874c29c9d8d30447,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,3.0,2018-06-20,2018-06-23 12:33:50,delivered,2018-01-25 20:59:12,2018-01-26 17:49:06,2018-01-29 21:19:11,2018-02-09,NaN,3
55943,7f39ba4c9052be115350065d07583cac,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,2018-06-19,2018-06-21 01:37:12,delivered,2018-03-11 19:40:32,2018-03-14 18:14:50,2018-03-27 20:38:17,2018-04-03,NaN,3
32068,48ac23662de1f4a94e29e7f3452a85d9,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,112.0,46.08,5.0,2017-12-12,2017-12-18 04:28:50,delivered,2016-10-04 12:25:20,2016-10-25 14:16:48,2016-10-26 16:41:21,2016-11-24,informatica_acessorios,208
62827,8f7176f38b6ac3f5e924b9b05716440c,001cca7ae9ae17fb1caed9dfb1094831,98a8c2fa16d7239c606640f5555768e4,109.0,44.84,5.0,2017-12-13,2017-12-13 09:11:13,delivered,2016-10-08 11:02:40,2016-10-12 11:02:41,2016-10-28 14:11:26,2016-12-14,NaN,208


In [77]:
'''def vec_dt_replace(series, year=None, month=None, day=None):
    return pd.to_datetime(
        {'year': series.dt.year if year is None else year,
         'month': series.dt.month if month is None else month,
         'day': series.dt.day if day is None else day})

df['snapshot_date'] = vec_dt_replace(df['snapshot_date'], day=1)
df.head()'''

"def vec_dt_replace(series, year=None, month=None, day=None):\n    return pd.to_datetime(\n        {'year': series.dt.year if year is None else year,\n         'month': series.dt.month if month is None else month,\n         'day': series.dt.day if day is None else day})\n\ndf['snapshot_date'] = vec_dt_replace(df['snapshot_date'], day=1)\ndf.head()"

In [78]:
'''list(df.snapshot_date.dt.month + 6) '''

'list(df.snapshot_date.dt.month + 6) '

In [79]:
'''df['snapshot_date'] = df['order_approved_at'] + pd.DateOffset(months=6)
df['snapshot_date'] = df['snapshot_date'].map(lambda x: LAST_DATE if x >= LAST_DATE else x)
df.tail()'''

"df['snapshot_date'] = df['order_approved_at'] + pd.DateOffset(months=6)\ndf['snapshot_date'] = df['snapshot_date'].map(lambda x: LAST_DATE if x >= LAST_DATE else x)\ndf.tail()"

In [80]:
'''df.snapshot_date.value_counts()'''

'df.snapshot_date.value_counts()'

## Dias sem vender

In [81]:
df['days_without_sell'] = df.groupby(['seller_id'])['order_approved_at'].diff()
df['days_without_sell'] = df['days_without_sell'].dt.days
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,review_creation_date,review_answer_timestamp,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_category_name,qtd_vendas,days_without_sell
93696,d455a8cb295653b55abda06d434ab492,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,2018-07-28,2018-07-30 23:59:38,delivered,2016-10-10 16:02:37,2016-10-18 15:53:51,2016-10-26 16:35:46,2016-12-14,NaN,3,NaN
69082,9dc8d1a6f16f1b89874c29c9d8d30447,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,3.0,2018-06-20,2018-06-23 12:33:50,delivered,2018-01-25 20:59:12,2018-01-26 17:49:06,2018-01-29 21:19:11,2018-02-09,NaN,3,472.0
55943,7f39ba4c9052be115350065d07583cac,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,2018-06-19,2018-06-21 01:37:12,delivered,2018-03-11 19:40:32,2018-03-14 18:14:50,2018-03-27 20:38:17,2018-04-03,NaN,3,44.0
32068,48ac23662de1f4a94e29e7f3452a85d9,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,112.0,46.08,5.0,2017-12-12,2017-12-18 04:28:50,delivered,2016-10-04 12:25:20,2016-10-25 14:16:48,2016-10-26 16:41:21,2016-11-24,informatica_acessorios,208,NaN
62827,8f7176f38b6ac3f5e924b9b05716440c,001cca7ae9ae17fb1caed9dfb1094831,98a8c2fa16d7239c606640f5555768e4,109.0,44.84,5.0,2017-12-13,2017-12-13 09:11:13,delivered,2016-10-08 11:02:40,2016-10-12 11:02:41,2016-10-28 14:11:26,2016-12-14,NaN,208,3.0


In [82]:
df.groupby(['seller_id'])['order_approved_at'].diff()

93696                 NaT
69082   472 days 04:56:35
55943    44 days 22:41:20
32068                 NaT
62827     3 days 22:37:20
               ...       
83803    10 days 18:17:17
91715    42 days 03:06:37
53977    28 days 00:00:01
42778    10 days 10:59:54
93638    21 days 18:32:54
Name: order_approved_at, Length: 77462, dtype: timedelta64[ns]

## Média de dias sem vender (dias para vender)

In [83]:
df['days_to_sell'] = df.groupby('seller_id')['days_without_sell'].transform('mean')
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,review_creation_date,review_answer_timestamp,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_category_name,qtd_vendas,days_without_sell,days_to_sell
93696,d455a8cb295653b55abda06d434ab492,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,2018-07-28,2018-07-30 23:59:38,delivered,2016-10-10 16:02:37,2016-10-18 15:53:51,2016-10-26 16:35:46,2016-12-14,NaN,3,NaN,258.000000
69082,9dc8d1a6f16f1b89874c29c9d8d30447,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,3.0,2018-06-20,2018-06-23 12:33:50,delivered,2018-01-25 20:59:12,2018-01-26 17:49:06,2018-01-29 21:19:11,2018-02-09,NaN,3,472.0,258.000000
55943,7f39ba4c9052be115350065d07583cac,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,2018-06-19,2018-06-21 01:37:12,delivered,2018-03-11 19:40:32,2018-03-14 18:14:50,2018-03-27 20:38:17,2018-04-03,NaN,3,44.0,258.000000
32068,48ac23662de1f4a94e29e7f3452a85d9,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,112.0,46.08,5.0,2017-12-12,2017-12-18 04:28:50,delivered,2016-10-04 12:25:20,2016-10-25 14:16:48,2016-10-26 16:41:21,2016-11-24,informatica_acessorios,208,NaN,3.412903
62827,8f7176f38b6ac3f5e924b9b05716440c,001cca7ae9ae17fb1caed9dfb1094831,98a8c2fa16d7239c606640f5555768e4,109.0,44.84,5.0,2017-12-13,2017-12-13 09:11:13,delivered,2016-10-08 11:02:40,2016-10-12 11:02:41,2016-10-28 14:11:26,2016-12-14,NaN,208,3.0,3.412903


## Total de receitas em vendas

In [84]:
df['income'] = df.groupby('seller_id')['price'].transform('sum')
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,review_creation_date,review_answer_timestamp,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_category_name,qtd_vendas,days_without_sell,days_to_sell,income
93696,d455a8cb295653b55abda06d434ab492,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,2018-07-28,2018-07-30 23:59:38,delivered,2016-10-10 16:02:37,2016-10-18 15:53:51,2016-10-26 16:35:46,2016-12-14,NaN,3,NaN,258.000000,2685.00
69082,9dc8d1a6f16f1b89874c29c9d8d30447,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,3.0,2018-06-20,2018-06-23 12:33:50,delivered,2018-01-25 20:59:12,2018-01-26 17:49:06,2018-01-29 21:19:11,2018-02-09,NaN,3,472.0,258.000000,2685.00
55943,7f39ba4c9052be115350065d07583cac,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,2018-06-19,2018-06-21 01:37:12,delivered,2018-03-11 19:40:32,2018-03-14 18:14:50,2018-03-27 20:38:17,2018-04-03,NaN,3,44.0,258.000000,2685.00
32068,48ac23662de1f4a94e29e7f3452a85d9,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,112.0,46.08,5.0,2017-12-12,2017-12-18 04:28:50,delivered,2016-10-04 12:25:20,2016-10-25 14:16:48,2016-10-26 16:41:21,2016-11-24,informatica_acessorios,208,NaN,3.412903,16440.16
62827,8f7176f38b6ac3f5e924b9b05716440c,001cca7ae9ae17fb1caed9dfb1094831,98a8c2fa16d7239c606640f5555768e4,109.0,44.84,5.0,2017-12-13,2017-12-13 09:11:13,delivered,2016-10-08 11:02:40,2016-10-12 11:02:41,2016-10-28 14:11:26,2016-12-14,NaN,208,3.0,3.412903,16440.16


## Ticket médio

In [85]:
df['average_ticket'] = df.income/df.qtd_vendas
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,review_creation_date,review_answer_timestamp,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_category_name,qtd_vendas,days_without_sell,days_to_sell,income,average_ticket
93696,d455a8cb295653b55abda06d434ab492,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,2018-07-28,2018-07-30 23:59:38,delivered,2016-10-10 16:02:37,2016-10-18 15:53:51,2016-10-26 16:35:46,2016-12-14,NaN,3,NaN,258.000000,2685.00,895.000000
69082,9dc8d1a6f16f1b89874c29c9d8d30447,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,3.0,2018-06-20,2018-06-23 12:33:50,delivered,2018-01-25 20:59:12,2018-01-26 17:49:06,2018-01-29 21:19:11,2018-02-09,NaN,3,472.0,258.000000,2685.00,895.000000
55943,7f39ba4c9052be115350065d07583cac,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,2018-06-19,2018-06-21 01:37:12,delivered,2018-03-11 19:40:32,2018-03-14 18:14:50,2018-03-27 20:38:17,2018-04-03,NaN,3,44.0,258.000000,2685.00,895.000000
32068,48ac23662de1f4a94e29e7f3452a85d9,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,112.0,46.08,5.0,2017-12-12,2017-12-18 04:28:50,delivered,2016-10-04 12:25:20,2016-10-25 14:16:48,2016-10-26 16:41:21,2016-11-24,informatica_acessorios,208,NaN,3.412903,16440.16,79.039231
62827,8f7176f38b6ac3f5e924b9b05716440c,001cca7ae9ae17fb1caed9dfb1094831,98a8c2fa16d7239c606640f5555768e4,109.0,44.84,5.0,2017-12-13,2017-12-13 09:11:13,delivered,2016-10-08 11:02:40,2016-10-12 11:02:41,2016-10-28 14:11:26,2016-12-14,NaN,208,3.0,3.412903,16440.16,79.039231


## Média de receitas de vendas por mês

In [86]:
df['avg_month_income'] = (df['income']/
                         round(((df['order_approved_at'].iloc[-1] - df['order_approved_at'].
                                 iloc[0])/np.timedelta64(1, 'M'))))
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,review_creation_date,review_answer_timestamp,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_category_name,qtd_vendas,days_without_sell,days_to_sell,income,average_ticket,avg_month_income
93696,d455a8cb295653b55abda06d434ab492,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,2018-07-28,2018-07-30 23:59:38,delivered,2016-10-10 16:02:37,2016-10-18 15:53:51,2016-10-26 16:35:46,2016-12-14,NaN,3,NaN,258.000000,2685.00,895.000000,149.166667
69082,9dc8d1a6f16f1b89874c29c9d8d30447,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,3.0,2018-06-20,2018-06-23 12:33:50,delivered,2018-01-25 20:59:12,2018-01-26 17:49:06,2018-01-29 21:19:11,2018-02-09,NaN,3,472.0,258.000000,2685.00,895.000000,149.166667
55943,7f39ba4c9052be115350065d07583cac,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,2018-06-19,2018-06-21 01:37:12,delivered,2018-03-11 19:40:32,2018-03-14 18:14:50,2018-03-27 20:38:17,2018-04-03,NaN,3,44.0,258.000000,2685.00,895.000000,149.166667
32068,48ac23662de1f4a94e29e7f3452a85d9,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,112.0,46.08,5.0,2017-12-12,2017-12-18 04:28:50,delivered,2016-10-04 12:25:20,2016-10-25 14:16:48,2016-10-26 16:41:21,2016-11-24,informatica_acessorios,208,NaN,3.412903,16440.16,79.039231,913.342222
62827,8f7176f38b6ac3f5e924b9b05716440c,001cca7ae9ae17fb1caed9dfb1094831,98a8c2fa16d7239c606640f5555768e4,109.0,44.84,5.0,2017-12-13,2017-12-13 09:11:13,delivered,2016-10-08 11:02:40,2016-10-12 11:02:41,2016-10-28 14:11:26,2016-12-14,NaN,208,3.0,3.412903,16440.16,79.039231,913.342222


## Tempo para responder

In [87]:
df['time_to_answer'] = (df['review_answer_timestamp'] - df['review_creation_date']).dt.days
df = df.drop(columns=['review_answer_timestamp', 'review_creation_date'])
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_category_name,qtd_vendas,days_without_sell,days_to_sell,income,average_ticket,avg_month_income,time_to_answer
93696,d455a8cb295653b55abda06d434ab492,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2016-10-10 16:02:37,2016-10-18 15:53:51,2016-10-26 16:35:46,2016-12-14,NaN,3,NaN,258.000000,2685.00,895.000000,149.166667,2.0
69082,9dc8d1a6f16f1b89874c29c9d8d30447,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,3.0,delivered,2018-01-25 20:59:12,2018-01-26 17:49:06,2018-01-29 21:19:11,2018-02-09,NaN,3,472.0,258.000000,2685.00,895.000000,149.166667,3.0
55943,7f39ba4c9052be115350065d07583cac,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2018-03-11 19:40:32,2018-03-14 18:14:50,2018-03-27 20:38:17,2018-04-03,NaN,3,44.0,258.000000,2685.00,895.000000,149.166667,2.0
32068,48ac23662de1f4a94e29e7f3452a85d9,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,112.0,46.08,5.0,delivered,2016-10-04 12:25:20,2016-10-25 14:16:48,2016-10-26 16:41:21,2016-11-24,informatica_acessorios,208,NaN,3.412903,16440.16,79.039231,913.342222,6.0
62827,8f7176f38b6ac3f5e924b9b05716440c,001cca7ae9ae17fb1caed9dfb1094831,98a8c2fa16d7239c606640f5555768e4,109.0,44.84,5.0,delivered,2016-10-08 11:02:40,2016-10-12 11:02:41,2016-10-28 14:11:26,2016-12-14,NaN,208,3.0,3.412903,16440.16,79.039231,913.342222,0.0


## Vendas por mês

In [88]:
df['avg_month_sell'] = (df['qtd_vendas']/
                         round(((df['order_approved_at'].iloc[-1] - df['order_approved_at'].
                                 iloc[0])/np.timedelta64(1, 'M'))))
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_category_name,qtd_vendas,days_without_sell,days_to_sell,income,average_ticket,avg_month_income,time_to_answer,avg_month_sell
93696,d455a8cb295653b55abda06d434ab492,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2016-10-10 16:02:37,2016-10-18 15:53:51,2016-10-26 16:35:46,2016-12-14,NaN,3,NaN,258.000000,2685.00,895.000000,149.166667,2.0,0.166667
69082,9dc8d1a6f16f1b89874c29c9d8d30447,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,3.0,delivered,2018-01-25 20:59:12,2018-01-26 17:49:06,2018-01-29 21:19:11,2018-02-09,NaN,3,472.0,258.000000,2685.00,895.000000,149.166667,3.0,0.166667
55943,7f39ba4c9052be115350065d07583cac,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2018-03-11 19:40:32,2018-03-14 18:14:50,2018-03-27 20:38:17,2018-04-03,NaN,3,44.0,258.000000,2685.00,895.000000,149.166667,2.0,0.166667
32068,48ac23662de1f4a94e29e7f3452a85d9,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,112.0,46.08,5.0,delivered,2016-10-04 12:25:20,2016-10-25 14:16:48,2016-10-26 16:41:21,2016-11-24,informatica_acessorios,208,NaN,3.412903,16440.16,79.039231,913.342222,6.0,11.555556
62827,8f7176f38b6ac3f5e924b9b05716440c,001cca7ae9ae17fb1caed9dfb1094831,98a8c2fa16d7239c606640f5555768e4,109.0,44.84,5.0,delivered,2016-10-08 11:02:40,2016-10-12 11:02:41,2016-10-28 14:11:26,2016-12-14,NaN,208,3.0,3.412903,16440.16,79.039231,913.342222,0.0,11.555556


## Idade em meses

In [122]:
'''df['age'] = np.ceil(((LAST_DATE - df.groupby('seller_id')['order_approved_at']
                     .transform('min'))/np.timedelta64(1, 'M')))'''
df['age'] = np.ceil((LAST_DATE - df.groupby('seller_id')['order_approved_at']
                     .transform('min'))/np.timedelta64(30, 'D'))
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,...,avg_month_sell,age,avg_freight,distinct_cat,avg_estimated_time,avg_real_delivery_time,islate,month_that_sell,income_activation_months,act_prop
0,d455a8cb295653b55abda06d434ab492,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2016-10-10 16:02:37,2016-10-18 15:53:51,2016-10-26 16:35:46,...,0.166667,20.0,21.020000,0,29.333333,8.000000,0,3,895.000000,0.15
1,9dc8d1a6f16f1b89874c29c9d8d30447,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,3.0,delivered,2018-01-25 20:59:12,2018-01-26 17:49:06,2018-01-29 21:19:11,...,0.166667,20.0,21.020000,0,29.333333,8.000000,0,3,895.000000,0.15
2,7f39ba4c9052be115350065d07583cac,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2018-03-11 19:40:32,2018-03-14 18:14:50,2018-03-27 20:38:17,...,0.166667,20.0,21.020000,0,29.333333,8.000000,0,3,895.000000,0.15
3,48ac23662de1f4a94e29e7f3452a85d9,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,112.0,46.08,5.0,delivered,2016-10-04 12:25:20,2016-10-25 14:16:48,2016-10-26 16:41:21,...,11.555556,21.0,37.170897,61,21.301282,9.237179,0,18,913.342222,0.90
4,8f7176f38b6ac3f5e924b9b05716440c,001cca7ae9ae17fb1caed9dfb1094831,98a8c2fa16d7239c606640f5555768e4,109.0,44.84,5.0,delivered,2016-10-08 11:02:40,2016-10-12 11:02:41,2016-10-28 14:11:26,...,11.555556,21.0,37.170897,61,21.301282,9.237179,0,18,913.342222,0.90


In [123]:
df['age'].value_counts()

20.0    25721
17.0    21409
16.0    11002
21.0     7212
15.0     4580
14.0     2404
13.0     1499
12.0      942
11.0      578
10.0      499
9.0       355
7.0       333
8.0       278
6.0       183
4.0       118
5.0       117
3.0        95
2.0        73
1.0        64
Name: age, dtype: int64

## Média de frete

In [91]:
df['avg_freight'] = df.groupby('seller_id')['freight_value'].transform('mean')
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,...,qtd_vendas,days_without_sell,days_to_sell,income,average_ticket,avg_month_income,time_to_answer,avg_month_sell,age,avg_freight
93696,d455a8cb295653b55abda06d434ab492,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2016-10-10 16:02:37,2016-10-18 15:53:51,2016-10-26 16:35:46,...,3,NaN,258.000000,2685.00,895.000000,149.166667,2.0,0.166667,20.0,21.020000
69082,9dc8d1a6f16f1b89874c29c9d8d30447,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,3.0,delivered,2018-01-25 20:59:12,2018-01-26 17:49:06,2018-01-29 21:19:11,...,3,472.0,258.000000,2685.00,895.000000,149.166667,3.0,0.166667,20.0,21.020000
55943,7f39ba4c9052be115350065d07583cac,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2018-03-11 19:40:32,2018-03-14 18:14:50,2018-03-27 20:38:17,...,3,44.0,258.000000,2685.00,895.000000,149.166667,2.0,0.166667,20.0,21.020000
32068,48ac23662de1f4a94e29e7f3452a85d9,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,112.0,46.08,5.0,delivered,2016-10-04 12:25:20,2016-10-25 14:16:48,2016-10-26 16:41:21,...,208,NaN,3.412903,16440.16,79.039231,913.342222,6.0,11.555556,20.0,37.170897
62827,8f7176f38b6ac3f5e924b9b05716440c,001cca7ae9ae17fb1caed9dfb1094831,98a8c2fa16d7239c606640f5555768e4,109.0,44.84,5.0,delivered,2016-10-08 11:02:40,2016-10-12 11:02:41,2016-10-28 14:11:26,...,208,3.0,3.412903,16440.16,79.039231,913.342222,0.0,11.555556,20.0,37.170897


## Quantidade de categorias distintas

In [92]:
df['distinct_cat'] = df.groupby('seller_id')['product_category_name'].transform('count')
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,...,days_without_sell,days_to_sell,income,average_ticket,avg_month_income,time_to_answer,avg_month_sell,age,avg_freight,distinct_cat
93696,d455a8cb295653b55abda06d434ab492,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2016-10-10 16:02:37,2016-10-18 15:53:51,2016-10-26 16:35:46,...,NaN,258.000000,2685.00,895.000000,149.166667,2.0,0.166667,20.0,21.020000,0
69082,9dc8d1a6f16f1b89874c29c9d8d30447,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,3.0,delivered,2018-01-25 20:59:12,2018-01-26 17:49:06,2018-01-29 21:19:11,...,472.0,258.000000,2685.00,895.000000,149.166667,3.0,0.166667,20.0,21.020000,0
55943,7f39ba4c9052be115350065d07583cac,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2018-03-11 19:40:32,2018-03-14 18:14:50,2018-03-27 20:38:17,...,44.0,258.000000,2685.00,895.000000,149.166667,2.0,0.166667,20.0,21.020000,0
32068,48ac23662de1f4a94e29e7f3452a85d9,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,112.0,46.08,5.0,delivered,2016-10-04 12:25:20,2016-10-25 14:16:48,2016-10-26 16:41:21,...,NaN,3.412903,16440.16,79.039231,913.342222,6.0,11.555556,20.0,37.170897,61
62827,8f7176f38b6ac3f5e924b9b05716440c,001cca7ae9ae17fb1caed9dfb1094831,98a8c2fa16d7239c606640f5555768e4,109.0,44.84,5.0,delivered,2016-10-08 11:02:40,2016-10-12 11:02:41,2016-10-28 14:11:26,...,3.0,3.412903,16440.16,79.039231,913.342222,0.0,11.555556,20.0,37.170897,61


## Prazo estimado

In [93]:
df['avg_estimated_time'] = (df['order_estimated_delivery_date'] - df['order_delivered_carrier_date']).dt.days
df['avg_estimated_time'] = df.groupby('seller_id')['avg_estimated_time'].transform('mean')
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,...,days_to_sell,income,average_ticket,avg_month_income,time_to_answer,avg_month_sell,age,avg_freight,distinct_cat,avg_estimated_time
93696,d455a8cb295653b55abda06d434ab492,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2016-10-10 16:02:37,2016-10-18 15:53:51,2016-10-26 16:35:46,...,258.000000,2685.00,895.000000,149.166667,2.0,0.166667,20.0,21.020000,0,29.333333
69082,9dc8d1a6f16f1b89874c29c9d8d30447,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,3.0,delivered,2018-01-25 20:59:12,2018-01-26 17:49:06,2018-01-29 21:19:11,...,258.000000,2685.00,895.000000,149.166667,3.0,0.166667,20.0,21.020000,0,29.333333
55943,7f39ba4c9052be115350065d07583cac,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2018-03-11 19:40:32,2018-03-14 18:14:50,2018-03-27 20:38:17,...,258.000000,2685.00,895.000000,149.166667,2.0,0.166667,20.0,21.020000,0,29.333333
32068,48ac23662de1f4a94e29e7f3452a85d9,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,112.0,46.08,5.0,delivered,2016-10-04 12:25:20,2016-10-25 14:16:48,2016-10-26 16:41:21,...,3.412903,16440.16,79.039231,913.342222,6.0,11.555556,20.0,37.170897,61,21.301282
62827,8f7176f38b6ac3f5e924b9b05716440c,001cca7ae9ae17fb1caed9dfb1094831,98a8c2fa16d7239c606640f5555768e4,109.0,44.84,5.0,delivered,2016-10-08 11:02:40,2016-10-12 11:02:41,2016-10-28 14:11:26,...,3.412903,16440.16,79.039231,913.342222,0.0,11.555556,20.0,37.170897,61,21.301282


## Prazo real

In [94]:
df['avg_real_delivery_time'] = (df['order_delivered_customer_date'] - df['order_delivered_carrier_date']).dt.days
df['avg_real_delivery_time'] = df.groupby('seller_id')['avg_real_delivery_time'].transform('mean')
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,...,income,average_ticket,avg_month_income,time_to_answer,avg_month_sell,age,avg_freight,distinct_cat,avg_estimated_time,avg_real_delivery_time
93696,d455a8cb295653b55abda06d434ab492,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2016-10-10 16:02:37,2016-10-18 15:53:51,2016-10-26 16:35:46,...,2685.00,895.000000,149.166667,2.0,0.166667,20.0,21.020000,0,29.333333,8.000000
69082,9dc8d1a6f16f1b89874c29c9d8d30447,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,3.0,delivered,2018-01-25 20:59:12,2018-01-26 17:49:06,2018-01-29 21:19:11,...,2685.00,895.000000,149.166667,3.0,0.166667,20.0,21.020000,0,29.333333,8.000000
55943,7f39ba4c9052be115350065d07583cac,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2018-03-11 19:40:32,2018-03-14 18:14:50,2018-03-27 20:38:17,...,2685.00,895.000000,149.166667,2.0,0.166667,20.0,21.020000,0,29.333333,8.000000
32068,48ac23662de1f4a94e29e7f3452a85d9,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,112.0,46.08,5.0,delivered,2016-10-04 12:25:20,2016-10-25 14:16:48,2016-10-26 16:41:21,...,16440.16,79.039231,913.342222,6.0,11.555556,20.0,37.170897,61,21.301282,9.237179
62827,8f7176f38b6ac3f5e924b9b05716440c,001cca7ae9ae17fb1caed9dfb1094831,98a8c2fa16d7239c606640f5555768e4,109.0,44.84,5.0,delivered,2016-10-08 11:02:40,2016-10-12 11:02:41,2016-10-28 14:11:26,...,16440.16,79.039231,913.342222,0.0,11.555556,20.0,37.170897,61,21.301282,9.237179


## Atrasa?

In [95]:
df['islate'] = (df.avg_estimated_time - df.avg_real_delivery_time).map(lambda x: 1 if x < 0 else 0)
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,...,average_ticket,avg_month_income,time_to_answer,avg_month_sell,age,avg_freight,distinct_cat,avg_estimated_time,avg_real_delivery_time,islate
93696,d455a8cb295653b55abda06d434ab492,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2016-10-10 16:02:37,2016-10-18 15:53:51,2016-10-26 16:35:46,...,895.000000,149.166667,2.0,0.166667,20.0,21.020000,0,29.333333,8.000000,0
69082,9dc8d1a6f16f1b89874c29c9d8d30447,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,3.0,delivered,2018-01-25 20:59:12,2018-01-26 17:49:06,2018-01-29 21:19:11,...,895.000000,149.166667,3.0,0.166667,20.0,21.020000,0,29.333333,8.000000,0
55943,7f39ba4c9052be115350065d07583cac,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2018-03-11 19:40:32,2018-03-14 18:14:50,2018-03-27 20:38:17,...,895.000000,149.166667,2.0,0.166667,20.0,21.020000,0,29.333333,8.000000,0
32068,48ac23662de1f4a94e29e7f3452a85d9,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,112.0,46.08,5.0,delivered,2016-10-04 12:25:20,2016-10-25 14:16:48,2016-10-26 16:41:21,...,79.039231,913.342222,6.0,11.555556,20.0,37.170897,61,21.301282,9.237179,0
62827,8f7176f38b6ac3f5e924b9b05716440c,001cca7ae9ae17fb1caed9dfb1094831,98a8c2fa16d7239c606640f5555768e4,109.0,44.84,5.0,delivered,2016-10-08 11:02:40,2016-10-12 11:02:41,2016-10-28 14:11:26,...,79.039231,913.342222,0.0,11.555556,20.0,37.170897,61,21.301282,9.237179,0


## Qtd meses que vendeu

In [96]:
df2 = (df.groupby(['seller_id', df.order_approved_at.dt.year, df.order_approved_at.dt.month])).nunique()['order_approved_at'].rename_axis(['seller_id', 'year', 'month']).reset_index(name='qtd_sells')
df2.head()

,seller_id,year,month,qtd_sells
0,0015a82c2db000af6aaaf3ae2ecb0532,2016,10,1
1,0015a82c2db000af6aaaf3ae2ecb0532,2018,1,1
2,0015a82c2db000af6aaaf3ae2ecb0532,2018,3,1
3,001cca7ae9ae17fb1caed9dfb1094831,2016,10,2
4,001cca7ae9ae17fb1caed9dfb1094831,2017,1,3


In [97]:
df2 = df2.groupby(['seller_id'])['month'].count().reset_index(name='month_that_sell')
df = pd.merge(df, df2, how='inner', on='seller_id')
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,...,avg_month_income,time_to_answer,avg_month_sell,age,avg_freight,distinct_cat,avg_estimated_time,avg_real_delivery_time,islate,month_that_sell
0,d455a8cb295653b55abda06d434ab492,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2016-10-10 16:02:37,2016-10-18 15:53:51,2016-10-26 16:35:46,...,149.166667,2.0,0.166667,20.0,21.020000,0,29.333333,8.000000,0,3
1,9dc8d1a6f16f1b89874c29c9d8d30447,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,3.0,delivered,2018-01-25 20:59:12,2018-01-26 17:49:06,2018-01-29 21:19:11,...,149.166667,3.0,0.166667,20.0,21.020000,0,29.333333,8.000000,0,3
2,7f39ba4c9052be115350065d07583cac,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2018-03-11 19:40:32,2018-03-14 18:14:50,2018-03-27 20:38:17,...,149.166667,2.0,0.166667,20.0,21.020000,0,29.333333,8.000000,0,3
3,48ac23662de1f4a94e29e7f3452a85d9,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,112.0,46.08,5.0,delivered,2016-10-04 12:25:20,2016-10-25 14:16:48,2016-10-26 16:41:21,...,913.342222,6.0,11.555556,20.0,37.170897,61,21.301282,9.237179,0,18
4,8f7176f38b6ac3f5e924b9b05716440c,001cca7ae9ae17fb1caed9dfb1094831,98a8c2fa16d7239c606640f5555768e4,109.0,44.84,5.0,delivered,2016-10-08 11:02:40,2016-10-12 11:02:41,2016-10-28 14:11:26,...,913.342222,0.0,11.555556,20.0,37.170897,61,21.301282,9.237179,0,18


## Receita nos meses que vendeu

In [98]:
df['income_activation_months'] = df['income']/df['month_that_sell']
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,...,time_to_answer,avg_month_sell,age,avg_freight,distinct_cat,avg_estimated_time,avg_real_delivery_time,islate,month_that_sell,income_activation_months
0,d455a8cb295653b55abda06d434ab492,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2016-10-10 16:02:37,2016-10-18 15:53:51,2016-10-26 16:35:46,...,2.0,0.166667,20.0,21.020000,0,29.333333,8.000000,0,3,895.000000
1,9dc8d1a6f16f1b89874c29c9d8d30447,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,3.0,delivered,2018-01-25 20:59:12,2018-01-26 17:49:06,2018-01-29 21:19:11,...,3.0,0.166667,20.0,21.020000,0,29.333333,8.000000,0,3,895.000000
2,7f39ba4c9052be115350065d07583cac,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2018-03-11 19:40:32,2018-03-14 18:14:50,2018-03-27 20:38:17,...,2.0,0.166667,20.0,21.020000,0,29.333333,8.000000,0,3,895.000000
3,48ac23662de1f4a94e29e7f3452a85d9,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,112.0,46.08,5.0,delivered,2016-10-04 12:25:20,2016-10-25 14:16:48,2016-10-26 16:41:21,...,6.0,11.555556,20.0,37.170897,61,21.301282,9.237179,0,18,913.342222
4,8f7176f38b6ac3f5e924b9b05716440c,001cca7ae9ae17fb1caed9dfb1094831,98a8c2fa16d7239c606640f5555768e4,109.0,44.84,5.0,delivered,2016-10-08 11:02:40,2016-10-12 11:02:41,2016-10-28 14:11:26,...,0.0,11.555556,20.0,37.170897,61,21.301282,9.237179,0,18,913.342222


## Proporção de ativação

In [124]:
df['act_prop'] = df.month_that_sell/df.age
df.head()

,order_id,seller_id,product_id,price,freight_value,review_score,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,...,avg_month_sell,age,avg_freight,distinct_cat,avg_estimated_time,avg_real_delivery_time,islate,month_that_sell,income_activation_months,act_prop
0,d455a8cb295653b55abda06d434ab492,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2016-10-10 16:02:37,2016-10-18 15:53:51,2016-10-26 16:35:46,...,0.166667,20.0,21.020000,0,29.333333,8.000000,0,3,895.000000,0.150000
1,9dc8d1a6f16f1b89874c29c9d8d30447,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,3.0,delivered,2018-01-25 20:59:12,2018-01-26 17:49:06,2018-01-29 21:19:11,...,0.166667,20.0,21.020000,0,29.333333,8.000000,0,3,895.000000,0.150000
2,7f39ba4c9052be115350065d07583cac,0015a82c2db000af6aaaf3ae2ecb0532,a2ff5a97bf95719e38ea2e3b4105bce8,895.0,21.02,5.0,delivered,2018-03-11 19:40:32,2018-03-14 18:14:50,2018-03-27 20:38:17,...,0.166667,20.0,21.020000,0,29.333333,8.000000,0,3,895.000000,0.150000
3,48ac23662de1f4a94e29e7f3452a85d9,001cca7ae9ae17fb1caed9dfb1094831,0da9ffd92214425d880de3f94e74ce39,112.0,46.08,5.0,delivered,2016-10-04 12:25:20,2016-10-25 14:16:48,2016-10-26 16:41:21,...,11.555556,21.0,37.170897,61,21.301282,9.237179,0,18,913.342222,0.857143
4,8f7176f38b6ac3f5e924b9b05716440c,001cca7ae9ae17fb1caed9dfb1094831,98a8c2fa16d7239c606640f5555768e4,109.0,44.84,5.0,delivered,2016-10-08 11:02:40,2016-10-12 11:02:41,2016-10-28 14:11:26,...,11.555556,21.0,37.170897,61,21.301282,9.237179,0,18,913.342222,0.857143


In [128]:
df['act_prop'].value_counts()

0.900000    21523
1.000000    18058
0.857143     6374
0.941176     4022
0.850000     2197
            ...  
0.888889       10
0.058824        9
0.238095        5
0.095238        2
0.050000        1
Name: act_prop, Length: 108, dtype: int64

Termino aqui a criação do book de variáveis. A partir daqui focarei em criar as safras e finalmente cirar a analytical